In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as Fk
from torch_geometric.nn import DeepGraphInfomax, GCNConv
from torch_geometric.datasets import Planetoid
import numpy as np

In [2]:
dataset = Planetoid(root='/tmp/Citeseer', name='Citeseer')
data = dataset[0]

Processing...
Done!


In [6]:
class GCN(nn.Module):
    def __init__(self, ft_in, n_fts):
        super(GCN, self).__init__()
        self.conv = GCNConv(ft_in, n_fts)
        self.act = nn.PReLU(n_fts)
        
    def forward(self, x, edge_index):
        x = self.conv(x, edge_index)
        x = self.act(x)
        return x

In [7]:
def corruption(x, edge_index):
    return x[torch.randperm(x.size(0))], edge_index

In [8]:
def summary(h, *args, **kwargs):
    return torch.mean(h, dim=0)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepGraphInfomax(hidden_channels=512, 
                         encoder=GCN(data.num_features, 512),
                         summary=summary, 
                         corruption=corruption).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [10]:
def train():
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(data.x, data.edge_index)
    loss = model.loss(pos_z, neg_z, summary)
    loss.backward()
    optimizer.step()
    return loss.item()

In [11]:
def test():
    model.eval()
    z, _, _ = model(data.x, data.edge_index)
    acc = model.test(
    z[data.train_mask],data.y[data.train_mask],z[data.test_mask],data.y[data.test_mask],max_iter=10)
    return acc

In [12]:
for epoch in range(100):
    loss = train()
    if ((epoch+1) % 10) == 0:
        print("Epoch: {:d}, Loss: {:.4f}".format(epoch+1, loss))
acc = test()
print("Accuracy: {:.4f}".format(acc))

Epoch: 10, Loss: 1.2206
Epoch: 20, Loss: 0.8322
Epoch: 30, Loss: 0.4920
Epoch: 40, Loss: 0.3300
Epoch: 50, Loss: 0.2627
Epoch: 60, Loss: 0.2438
Epoch: 70, Loss: 0.2997
Epoch: 80, Loss: 0.2786
Epoch: 90, Loss: 0.3120
Epoch: 100, Loss: 0.2219
Accuracy: 0.7140


C:\Users\ahn\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
